In [1]:
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random as rn
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import codecs
from keras.layers.convolutional import Conv2D

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

In [4]:
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
#vad = np.load('../../data/y_egemaps.npy')
# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

# load model
EMBEDDING_DIM = 300
file_loc = '../../dimensionalSM/data/crawl-300d-2M-subword.vec'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))


earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)


Found 3437 unique tokens
../../dimensionalSM/data/crawl-300d-2M-subword.vec
G Word embeddings: 2000000
G Null word embeddings: 133


In [5]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [6]:
# split train/test
split = 7869

In [7]:
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
    
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(10039, 200, 189, 1)

In [8]:
# API model
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)

    #mocap network
    input_mocap = Input(shape=(200,189,1))
    model_mocap1 = Conv2D(64, 3, strides=(3, 3), padding="same")(input_mocap)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = Conv2D(256, 3,strides=(3, 3), padding="same")(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap9)   
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = Conv2D(64, 3,strides=(3, 3), padding="same")(model_mocap12)  
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    # combined model
    model_combined = concatenate([model_speech, model_text, model_mocap])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_text, input_mocap], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model_1 = model()
# model_1.summary()

In [9]:
# main function for for LOSO
def main():
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:split],x_train_text[:split],x_train_mocap[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[split:],x_train_text[split:],x_train_mocap[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(30):
        best = main()
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('stmFASTTEXT_multi7869-3.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 71s 11ms/step - loss: 1.9878 - v_loss: 0.7200 - a_loss: 0.6012 - d_loss: 0.6652 - v_ccc: 0.2793 - a_ccc: 0.3984 - d_ccc: 0.3344 - val_loss: 2.0448 - val_v_loss: 0.7167 - val_a_loss: 0.7173 - val_d_loss: 0.6343 - val_v_ccc: 0.2908 - val_a_ccc: 0.2887 - val_d_ccc: 0.3757
Epoch 2/50
6295/6295 [==============================] - 63s 10ms/step - loss: 1.1326 - v_loss: 0.3862 - a_loss: 0.3260 - d_loss: 0.4200 - v_ccc: 0.6137 - a_ccc: 0.6739 - d_ccc: 0.5798 - val_loss: 1.6127 - val_v_loss: 0.5839 - val_a_loss: 0.4941 - val_d_loss: 0.5453 - val_v_ccc: 0.4202 - val_a_ccc: 0.5115 - val_d_ccc: 0.4557
Epoch 3/50
6295/6295 [==============================] - 63s 10ms/step - loss: 1.0021 - v_loss: 0.3110 - a_loss: 0.3069 - d_loss: 0.3840 - v_ccc: 0.6889 - a_ccc: 0.6931 - d_ccc: 0.6158 - val_loss: 1.5736 - val_v_loss: 0

Epoch 10/50
6295/6295 [==============================] - 63s 10ms/step - loss: 0.6159 - v_loss: 0.1574 - a_loss: 0.2154 - d_loss: 0.2434 - v_ccc: 0.8427 - a_ccc: 0.7848 - d_ccc: 0.7567 - val_loss: 1.5051 - val_v_loss: 0.5235 - val_a_loss: 0.4367 - val_d_loss: 0.5610 - val_v_ccc: 0.4844 - val_a_ccc: 0.5659 - val_d_ccc: 0.4446
Epoch 11/50
6295/6295 [==============================] - 63s 10ms/step - loss: 0.5745 - v_loss: 0.1485 - a_loss: 0.1997 - d_loss: 0.2261 - v_ccc: 0.8514 - a_ccc: 0.8003 - d_ccc: 0.7739 - val_loss: 1.5671 - val_v_loss: 0.5109 - val_a_loss: 0.4715 - val_d_loss: 0.5945 - val_v_ccc: 0.4927 - val_a_ccc: 0.5280 - val_d_ccc: 0.4122
Epoch 12/50
6295/6295 [==============================] - 63s 10ms/step - loss: 0.5528 - v_loss: 0.1394 - a_loss: 0.1956 - d_loss: 0.2178 - v_ccc: 0.8605 - a_ccc: 0.8045 - d_ccc: 0.7822 - val_loss: 1.5310 - val_v_loss: 0.5003 - val_a_loss: 0.4553 - val_d_loss: 0.5928 - val_v_ccc: 0.5076 - val_a_ccc: 0.5476 - val_d_ccc: 0.4138
Epoch 13/50
6295/62

Epoch 19/50
6295/6295 [==============================] - 62s 10ms/step - loss: 0.3650 - v_loss: 0.0976 - a_loss: 0.1284 - d_loss: 0.1392 - v_ccc: 0.9025 - a_ccc: 0.8717 - d_ccc: 0.8609 - val_loss: 1.5738 - val_v_loss: 0.5308 - val_a_loss: 0.4939 - val_d_loss: 0.5657 - val_v_ccc: 0.4736 - val_a_ccc: 0.5139 - val_d_ccc: 0.4388
Epoch 20/50
6295/6295 [==============================] - 61s 10ms/step - loss: 0.3515 - v_loss: 0.0955 - a_loss: 0.1238 - d_loss: 0.1319 - v_ccc: 0.9044 - a_ccc: 0.8761 - d_ccc: 0.8680 - val_loss: 1.5799 - val_v_loss: 0.5346 - val_a_loss: 0.4962 - val_d_loss: 0.5621 - val_v_ccc: 0.4726 - val_a_ccc: 0.5097 - val_d_ccc: 0.4378
Epoch 21/50
6295/6295 [==============================] - 61s 10ms/step - loss: 0.3356 - v_loss: 0.0887 - a_loss: 0.1199 - d_loss: 0.1269 - v_ccc: 0.9112 - a_ccc: 0.8801 - d_ccc: 0.8730 - val_loss: 1.5881 - val_v_loss: 0.5310 - val_a_loss: 0.4855 - val_d_loss: 0.5798 - val_v_ccc: 0.4719 - val_a_ccc: 0.5194 - val_d_ccc: 0.4206
Epoch 22/50
2170/21

6295/6295 [==============================] - 63s 10ms/step - loss: 0.9008 - v_loss: 0.2662 - a_loss: 0.2857 - d_loss: 0.3496 - v_ccc: 0.7341 - a_ccc: 0.7144 - d_ccc: 0.6508 - val_loss: 1.5556 - val_v_loss: 0.5904 - val_a_loss: 0.4401 - val_d_loss: 0.5367 - val_v_ccc: 0.4121 - val_a_ccc: 0.5631 - val_d_ccc: 0.4692
Epoch 5/50
6295/6295 [==============================] - 63s 10ms/step - loss: 0.8405 - v_loss: 0.2367 - a_loss: 0.2745 - d_loss: 0.3294 - v_ccc: 0.7635 - a_ccc: 0.7254 - d_ccc: 0.6705 - val_loss: 1.5019 - val_v_loss: 0.5536 - val_a_loss: 0.4359 - val_d_loss: 0.5292 - val_v_ccc: 0.4549 - val_a_ccc: 0.5665 - val_d_ccc: 0.4767
Epoch 6/50
6295/6295 [==============================] - 63s 10ms/step - loss: 0.7860 - v_loss: 0.2178 - a_loss: 0.2621 - d_loss: 0.3058 - v_ccc: 0.7822 - a_ccc: 0.7378 - d_ccc: 0.6940 - val_loss: 1.4914 - val_v_loss: 0.5398 - val_a_loss: 0.4256 - val_d_loss: 0.5447 - val_v_ccc: 0.4655 - val_a_ccc: 0.5824 - val_d_ccc: 0.4607
Epoch 7/50
6295/6295 [===========

Epoch 13/50
6295/6295 [==============================] - 63s 10ms/step - loss: 0.4930 - v_loss: 0.1278 - a_loss: 0.1710 - d_loss: 0.1941 - v_ccc: 0.8722 - a_ccc: 0.8289 - d_ccc: 0.8059 - val_loss: 1.6258 - val_v_loss: 0.5478 - val_a_loss: 0.5001 - val_d_loss: 0.5914 - val_v_ccc: 0.4595 - val_a_ccc: 0.5016 - val_d_ccc: 0.4131
Epoch 14/50
6295/6295 [==============================] - 64s 10ms/step - loss: 0.4781 - v_loss: 0.1243 - a_loss: 0.1699 - d_loss: 0.1836 - v_ccc: 0.8756 - a_ccc: 0.8299 - d_ccc: 0.8163 - val_loss: 1.5404 - val_v_loss: 0.5218 - val_a_loss: 0.4806 - val_d_loss: 0.5543 - val_v_ccc: 0.4897 - val_a_ccc: 0.5187 - val_d_ccc: 0.4511
Epoch 15/50
6295/6295 [==============================] - 63s 10ms/step - loss: 0.4490 - v_loss: 0.1181 - a_loss: 0.1595 - d_loss: 0.1713 - v_ccc: 0.8818 - a_ccc: 0.8405 - d_ccc: 0.8286 - val_loss: 1.5456 - val_v_loss: 0.5334 - val_a_loss: 0.4791 - val_d_loss: 0.5461 - val_v_ccc: 0.4775 - val_a_ccc: 0.5190 - val_d_ccc: 0.4579
Epoch 16/50
6295/62

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 74s 12ms/step - loss: 0.7171 - v_loss: 0.1828 - a_loss: 0.2491 - d_loss: 0.2853 - v_ccc: 0.8173 - a_ccc: 0.7510 - d_ccc: 0.7146 - val_loss: 1.5259 - val_v_loss: 0.5290 - val_a_loss: 0.4310 - val_d_loss: 0.5831 - val_v_ccc: 0.4809 - val_a_ccc: 0.5702 - val_d_ccc: 0.4230
Epoch 8/50
6295/6295 [==============================] - 74s 12ms/step - loss: 0.6711 - v_loss: 0.1702 - a_loss: 0.2336 - d_loss: 0.2670 - v_ccc: 0.8298 - a_ccc: 0.7662 - d_ccc: 0.7329 - val_loss: 1.4973 - val_v_loss: 0.5151 - val_a_loss: 0.4338 - val_d_loss: 0.5627 - val_v_ccc: 0.4921 - val_a_ccc: 0.5671 - val_d_ccc: 0.4435
Epoch 9/50
6295/6295 [==============================] - 74s 12ms/step - loss: 0.6476 - v_loss: 0.1654 - a_loss: 0.2273 - d_loss: 0.2553 - v_ccc: 0.8346 - a_ccc: 0.7729 - d_ccc: 0.7449 - val_loss: 1.5804 - val_v_loss: 0.5365 - val_a_loss: 0.4431 - val_d_loss: 0.6151 - val_v_ccc: 0.4724 - val_a_ccc: 0.5591 - val_d_ccc: 0.3881
Epoch 10/50
6295/6295 [==========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2170/2170 [==============================] - 6s 3ms/step
[0.4368301331996918, 0.5897302031517029, 0.5056195855140686]
28
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 91s 14ms/step - loss: 1.9896 - v_loss: 0.7326 - a_loss: 0.5880 - d_loss: 0.6683 - v_ccc: 0.2672 - a_ccc: 0.4117 - d_ccc: 0.3314 - val_loss: 2.0332 - val_v_loss: 0.7299 - val_a_loss: 0.6788 - val_d_loss: 0.6482 - val_v_ccc: 0.2769 - val_a_ccc: 0.3271 - val_d_ccc: 0.3627
Epoch 2/50
6295/6295 [==============================] - 75s 12ms/step - loss: 1.1337 - v_loss: 0.3838 - a_loss: 0.3295 - d_loss: 0.4207 - v_ccc: 0.6161 - a_ccc: 0.6707 - d_ccc: 0.5795 - val_loss: 1.6578 - val_v_loss: 0.6406 - val_a_loss: 0.4854 - val_d_loss: 0.5451 - val_v_ccc: 0.3657 - val_a_ccc: 0.5207 - val_d_ccc: 0.4557
Epoch 3/50
6295/6295 [==============================] - 75s 12ms/step - loss: 0.9903 - v_loss: 0.3045 - a_loss: 0.3054 - d_loss: 0.3809 - v_ccc: 0.6956 - a_ccc: 0.6948 - d_ccc: 0.

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

